# 消息格式处理

Meesages 类通过对字符串、模板、元组等类型的兼容，以及默认的 role 生成规则，来快速构造消息列表。

大模型所需要的消息列表格式，通常是这样：
```python
[{'role': 'system', 'content': '你是一个小说家。'},
 {'role': 'user', 'content': '帮我创作吧'},
 {'role': 'assistant', 'content': '从前有一个人很坏，他坏死了。\n额，我是他说真的死了。'}]
 ```

这有些啰嗦，可以这样来简化：
```python
Messages([
    '你是一个小说家。',
    '帮我创作吧',
    '从前有一个人很坏，他坏死了。\n额，我是他说真的死了。'
])
 ```

In [2]:
from illufly.types import Message, Messages, Template

## role 生成规则

### 规则1： 不指定时，首条是 system，system 后接 user，user 与 assistant 交替

In [3]:
Messages(["hi"]).to_list()

[{'role': 'system', 'content': 'hi'}]

In [4]:
Messages(["hi", "我很开心", "我也很开心"]).to_list()

[{'role': 'system', 'content': 'hi'},
 {'role': 'user', 'content': '我很开心'},
 {'role': 'assistant', 'content': '我也很开心'}]

In [5]:
Messages([
    Template(template_text="你是一个小说家。"),
    "帮我创作吧",
    "从前有一个人很坏，他坏死了。\n额，我是他说真的死了。"
]).to_list()

[{'role': 'system', 'content': '你是一个小说家。'},
 {'role': 'user', 'content': '帮我创作吧'},
 {'role': 'assistant', 'content': '从前有一个人很坏，他坏死了。\n额，我是他说真的死了。'}]

In [6]:
Messages([
    ("system", Template(template_text="你是一个小说家，帮我创作吧。")),
    "我这个小说很有趣哦，我结束了",
]).to_list()

[{'role': 'system', 'content': '你是一个小说家，帮我创作吧。'},
 {'role': 'user', 'content': '我这个小说很有趣哦，我结束了'}]

In [7]:
Messages([
    ("user", Template(template_text="你是一个小说家，帮我创作一个说。")),
    {"role": "assistant", "content": "我开始了"},
]).to_list()

[{'role': 'user', 'content': '你是一个小说家，帮我创作一个说。'},
 {'role': 'assistant', 'content': '我开始了'}]

### 规则2：列表中允许出现字符串、Turple、Template、Message、Dict等类型

In [8]:
Messages([
    Template(template_text="你是一个小说家，帮我创作吧。"),
    ("user", "不错"),
    {"role": "assistant", "content": "请你开始"},
    "我这个小说很有趣哦，我结束了",
]).to_list()

[{'role': 'system', 'content': '你是一个小说家，帮我创作吧。'},
 {'role': 'user', 'content': '不错'},
 {'role': 'assistant', 'content': '请你开始'},
 {'role': 'user', 'content': '我这个小说很有趣哦，我结束了'}]

### 规则3：允许用 ai 替代 assistant 作为 role 来声明，但会被替换为 assistant

In [9]:
Messages([
    ("user", Template(template_text="你是一个小说家，帮我创作一个说。")),
    ("ai", "我开始了"),
    {"role": "user", "content": "我开始了"},
    {"role": "ai", "content": "我开始了"},
]).to_list()

[{'role': 'user', 'content': '你是一个小说家，帮我创作一个说。'},
 {'role': 'assistant', 'content': '我开始了'},
 {'role': 'user', 'content': '我开始了'},
 {'role': 'assistant', 'content': '我开始了'}]

## 提示语模板动态填充

使用 Template 可以根据映射规则，动态提取数据，完成模板组装。

In [1]:
from illufly.types import Template, Messages

t = Template("IDEA")
t.using_vars_list

['task']

### 模板输入变量映射

In [2]:
# 默认映射
Template("IDEA").input_mapping

{'task': 'task'}

In [3]:
# 指定映射
Template("IDEA", input_mapping={"task": "question"}).input_mapping

{'task': 'question'}

In [4]:
# 映射到嵌套字典
Template("IDEA", input_mapping={"task": "state.question"}).input_mapping

{'task': 'state.question'}

### 填充模板变量

In [5]:
# 格式化
Template("IDEA").format({"task": "请帮我写一首歌"})

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n请帮我写一首歌\n'

In [7]:
# 映射变量缺失
Template("IDEA")(verbose=True)

  0s [INFO] 你是强大的写作助手。

你必须遵循以下约束来完成任务:
1. 直接输出你的结果，不要评论，不要啰嗦
2. 使用markdown格式输出

**你的任务是:**




'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n\n'

In [8]:
# 改变映射规则
Template(
    "IDEA",
    input_mapping={"task": "question"}
).format({"question": "请帮我写一首歌"})

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n请帮我写一首歌\n'

In [9]:
# 映射到深层嵌套的字典变量
Template(
    "IDEA",
    input_mapping={"task": "state.my.question"}
).format({"state": {"my": {"question": "请帮我写一首歌"}}})

'你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n请帮我写一首歌\n'

In [10]:
messages = Messages(["hi", '我想回家', "有什么可以帮您?"])
messages.input_vars

[]

### 